In [ ]:
## Notebook to evaluate and store the results from ggcnn for a sample imageset
## Last Trial Run: 7th April 2021

In [6]:
import os 
import glob
import sys 
import torch
import numpy as np
import torch.utils.data 
import imageio
import pandas as pd
import time

In [7]:
ggcnn_dir = "/media/gvk/GVK_Elements/WPI/Spring2021/DR_Grasping/CurrentFocus/ggcnn_repo/"
fadi_dir = "/media/gvk/GVK_Elements/github_repo/Ensemble/Experts/Generative/pretrained_weights"
sys.path.append(ggcnn_dir)
sys.path.append(fadi_dir)
from models.common import post_process_output
from utils.dataset_processing import evaluation, grasp 
from utils.data import get_dataset
from utils.dataset_processing.image import DepthImage

In [8]:
## Considering one folder!
dataset_path = "/media/gvk/GVK_Elements/WPI/Spring2021/DR_Grasping/CurrentFocus/Cornell_Dataset/Full_Cornell/*/"
dataset = get_dataset("cornell")
sys.path.insert(0, dataset_path)
test_data = dataset(dataset_path, include_depth = True)


Search Path:  /media/gvk/GVK_Elements/WPI/Spring2021/DR_Grasping/CurrentFocus/Cornell_Dataset/Full_Cornell/*/*/pcd*cpos.txt


In [4]:
# ## Considering one folder!
# folder1_path = "/media/gvk/GVK_Elements/WPI/Spring2021/DR_Grasping/CurrentFocus/Cornell_Dataset/Full_Cornell/data01/"
# dataset = get_dataset("cornell")
# sys.path.insert(0, folder1_path)
# test_data = dataset(folder1_path, include_depth = True)


Search Path:  /media/gvk/GVK_Elements/WPI/Spring2021/DR_Grasping/CurrentFocus/Cornell_Dataset/Full_Cornell/data01/*/pcd*cpos.txt


In [9]:
len(test_data.depth_files)

885

In [10]:
## Model Loading:
fadi_depth_dir = "/media/gvk/GVK_Elements/github_repo/Stacking_Cornell/Experts/Generative/pretrained_weights/depth/ggcnn_epoch_23_cornell"
depth_net = torch.load(fadi_depth_dir)
net = depth_net

/media/gvk/GVK_Elements/py_linux/ensemble_dr_linux/lib/python3.8/site-packages/torch/serialization.py:656: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/media/gvk/GVK_Elements/py_linux/ensemble_dr_linux/lib/python3.8/site-packages/torch/serialization.py:656: SourceChangeWarning: source code of class 'torch.nn.modules.conv.ConvTranspose2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [11]:
test_comp = test_data
test_data = torch.utils.data.DataLoader(test_data, batch_size = 1, shuffle = False)

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [9]:
# ##  Trial for each test image
# start_cell = time.time()
# results = {'correct': 0, 'failed': 0}
# iou_quality = {}
# with torch.no_grad():

#     ## For each test image
#     ## Just testing for one single image
#     x, y, didx, rot, zoom =  next(iter(test_data))
#     idx = 0
#     xc = x.to(device)
#     yc = [yi.to(device) for yi in y]
#     lossd = net.compute_loss(xc, yc)    ## Ensure that your outputs are from a dataloader!

#     q_img, ang_img, width_img = post_process_output( lossd['pred']['pos'], lossd['pred']['cos'], lossd['pred']['sin'], lossd['pred']['width'])
#     s = evaluation.calculate_iou_match(q_img, ang_img, test_data.dataset.get_gtbb(didx, rot, zoom), no_grasps = 1, grasp_width = width_img)

#     if s:
#         results['correct'] += 1
#     else:
#         results['failed'] += 1

#     iou_dict = {}
#     ## Now to compare and create a dictionary that stores the quality given image name
#     x_comp, _, _, _, _ = test_comp[idx]
#     #print(torch.allclose(torch.tensor(x_comp), x))
#     if torch.allclose(torch.tensor(x_comp), x):
#         img_id = test_comp.depth_files[0].split("/")[-1]

#         ## We cannot use an image!
#         #quality_dict[str(img_id)] = q_img
#         image_id = str(img_id).split('.')[0][:-1]
#         iou_quality[image_id] = [int(s), np.max(q_img).astype('float32')]
#     else:
#         print("Dataloader and Comparison X didn't match")

# print("Cell Time: ", time.time() - start_cell)

Cell Time:  0.21756196022033691


In [13]:
results = {'correct': 0, 'failed': 0}
start_cell = time.time()
iou_quality = {}
with torch.no_grad():

    ## For each test image
    ## Just testing for one single image
    #x, y, didx, rot, zoom =  next(iter(test_data))
    for idx, (x, y, didx, rot, zoom) in enumerate(test_data):
        xc = x.to(device)
        yc = [yi.to(device) for yi in y]
        lossd = net.compute_loss(xc, yc)

        q_img, ang_img, width_img = post_process_output( lossd['pred']['pos'], lossd['pred']['cos'], lossd['pred']['sin'], lossd['pred']['width'])
        s = evaluation.calculate_iou_match(q_img, ang_img, test_data.dataset.get_gtbb(didx, rot, zoom), no_grasps = 1, grasp_width = width_img)

        if s:
            results['correct'] += 1
        else:
            results['failed'] += 1

        ## Need to store: q_img and img_id
        ## Now to compare and create a dictionary that stores the quality given image name
        x_comp, _, _, _, _ = test_comp[idx]
        
        ## Now, trying to 
        if torch.allclose(torch.tensor(x_comp), x):
            img_id = test_comp.depth_files[idx].split("/")[-1]
            img_id = str(img_id).split('.')[0][:-1]
            ## We cannot use an image!
            #quality_dict[str(img_id)] = q_img
            iou_quality[str(img_id)] = [int(s), np.max(q_img).astype('float32')]
        else:
            print("Dataloader and Comparison X didn't match")
print("Cell Time: ", time.time() - start_cell) ## ~104s

Cell Time:  103.87643551826477


In [14]:
print(len(iou_quality.keys()))

885


In [15]:
## To get the image id
img_id = test_comp.rgb_files[1].split("/")[-1]
depth_id = str(img_id).split('.')[0][:-1]
print(depth_id)

rgb_id = str(img_id).split('.')[0][:-1]
print(rgb_id)

pcd0101
pcd0101


In [16]:
## Saving the Collected Results!
import pickle
model_name = "fadi_depth"
save_dir = "/media/gvk/GVK_Elements/github_repo/Ensemble/Stacking_Test/Stacking_Data/"
first_entry, last_entry = list(iou_quality.keys())[0], list(iou_quality.keys())[-1]
fname = "_".join((model_name, first_entry, last_entry+".txt"))
fdir = os.path.join(save_dir, fname)
with open(fdir, 'wb') as file_pickle:
    pickle.dump(iou_quality, file_pickle)

In [17]:
fdir

'/media/gvk/GVK_Elements/github_repo/Ensemble/Stacking_Test/Stacking_Data/fadi_depth_pcd0100_pcd1034.txt'